In [5]:
from selenium import webdriver
import time,re,random,sys
import pandas as pd
import jieba.analyse

driver = webdriver.Chrome()
driver.set_window_size(800,800)

In [6]:
ordernm = -2
zhdf = pd.read_excel('zhmm.xlsx')
zhanghao,mima = zhdf.iloc[ordernm]['zhanghao'],zhdf.iloc[ordernm]['mima']
driver.get('https://passport.weibo.cn/signin/login?');time.sleep(1)
elem = driver.find_element_by_id('loginName')
elem.send_keys(zhanghao)
elem = driver.find_element_by_id('loginPassword')
elem.send_keys(mima)
driver.find_element_by_id('loginAction').click();time.sleep(5)

In [ ]:
topicname = '维多利亚的秘密'
homepage = r'https://weibo.cn/search/mblog?hideSearchFrame=&keyword=维多利亚的秘密&page='
    
weibodf = pd.Series()
for homelist in range(1,101,1):
    url = homepage+str(homelist)
    driver.get(url)
    time.sleep(2)
    df = pd.Series(re.findall(r'<span class="ctt">.*?赞',driver.page_source))
    df = df.str.replace(r'</span><br />',',')
    df = df.str.replace(r'<.*?>','')
    df = df.map(lambda x:''.join(re.findall(u'[\u4E00-\u9FA5，。？！ 0-9.￥a-z@#\[\]\*]',x)))
    df = df.str.replace('组图|\[图片\]|赞|共.*?张|显示地图|原图','')
    df = df.str.replace(r'\[\]','')
    df = df.str.replace(' |,,',',');df = df.str.replace(',,',',');df = df.str.replace(',,',',')
    
    weibodf = pd.concat([weibodf,df])

weibodf
weibodf.to_csv(topicname+'.csv',index=False,encoding='gb18030')

In [ ]:
import jieba.analyse
gjc=[]
for x in weibodf.str.replace('原图','').drop_duplicates().map(jieba.analyse.textrank):
    for y in x:
        gjc.append(y)
pd.Series(gjc).value_counts().to_excel('关键词-lulei.xlsx',encoding='gb18030')

In [ ]:
weibodf[weibodf.map(lambda x:'酸奶' in x)].drop_duplicates().to_excel('weibo-lulei.xlsx',index=False,encoding='gb18030')

In [ ]:
drive.close()

In [7]:
tasklist = '倩碧活芯乳'
tasklist = tasklist.split('；')

for i in tasklist:
    topicname = i
    homepage = r'https://weibo.cn/search/mblog?hideSearchFrame=&keyword=%s&page=' % i
    
    weibodf = pd.Series()
    for homelist in range(1,101,1):
        try:
            url = homepage+str(homelist)
            driver.get(url)
            time.sleep(2)
            df = pd.Series(re.findall(r'<span class="ctt">.*?赞',driver.page_source))
            df = df.str.replace(r'</span><br />',',')
            df = df.str.replace(r'<.*?>','')
            df = df.map(lambda x:''.join(re.findall(u'[\u4E00-\u9FA5，。？！ 0-9.￥a-z@#\[\]\*]',x)))
            df = df.str.replace('组图|\[图片\]|赞|共.*?张|显示地图|原图','')
            df = df.str.replace(r'\[\]','')
            df = df.str.replace(' |,,',',');df = df.str.replace(',,',',');df = df.str.replace(',,',',')
            weibodf = pd.concat([weibodf,df])
        except:
            pass
    
    rizhipath = i + '.xlsx'
    pd.DataFrame([]).to_excel(rizhipath)
    excelWriter = pd.ExcelWriter(rizhipath)
    
    weibodf.drop_duplicates().to_excel(excelWriter,index=False,sheet_name='原始数据')
    
    gjc=[]
    for x in weibodf.drop_duplicates().map(jieba.analyse.textrank):
        for y in x:
            gjc.append(y)
    
    pd.Series(gjc).value_counts().to_excel(excelWriter,sheet_name='词频统计')
    
    excelWriter.save()
    excelWriter.close()

In [4]:
driver.close()